In [3]:
pip install requests

Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install mysql-connector-python

Note: you may need to restart the kernel to use updated packages.


In [76]:
pip install json

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement json (from versions: none)
ERROR: No matching distribution found for json


In [136]:
pip install tkcalendar

In [181]:
pip install datetime


  Obtaining dependency information for datetime from https://files.pythonhosted.org/packages/95/88/3b9d4042b396221a132180b392ab2a174031a6fb579f7927f3909fc183a7/DateTime-5.2-py3-none-any.whl.metadata
     ---------------------------------------- 0.0/204.1 kB ? eta -:--:--
     ---------------------------------------- 0.0/204.1 kB ? eta -:--:--
     ------ --------------------------------- 30.7/204.1 kB ? eta -:--:--
     -------------- ---------------------- 81.9/204.1 kB 919.0 kB/s eta 0:00:01
     ------------------- ---------------- 112.6/204.1 kB 819.2 kB/s eta 0:00:01
     ------------------------- ---------- 143.4/204.1 kB 853.3 kB/s eta 0:00:01
     ---------------------------------- - 194.6/204.1 kB 841.6 kB/s eta 0:00:01
     ---------------------------------- - 194.6/204.1 kB 841.6 kB/s eta 0:00:01
     ------------------------------------ 204.1/204.1 kB 689.8 kB/s eta 0:00:00
  Obtaining dependency information for pytz from https://files.pythonhosted.org/packages/32/4d/aaf7e

In [199]:
import mysql.connector
mydb= mysql.connector.connect(
  host="localhost",
  user="root",
  password="",
  database='project'
  
)

print(mydb)
mycursor = mydb.cursor(buffered=True)

In [71]:
mycursor.execute("CREATE TABLE project.nasa(image_url VARCHAR(100),title VARCHAR(100),explanation VARCHAR(500),date DATE)")

In [279]:
import requests
import json
import tkinter as tk
from tkinter import ttk
from tkcalendar import Calendar,DateEntry
from datetime import datetime, timedelta

master = tk.Tk()
master.title("NASA")
master.geometry("850x300")

def extract_with_pattern(response_text, pattern):
    match = re.search(pattern, response_text)
    return match.group(1) if match else None

def handler():
    date_to_be_search = e2.get()
    formatted_Date = datetime.strptime(date_to_be_search, "%m/%d/%y").strftime("%Y-%m-%d")
    temp_date = datetime.strptime(formatted_Date, '%Y-%m-%d')
    start_date = temp_date - relativedelta(days=10)
    start_date = start_date.strftime("%Y-%m-%d")
    data_source = selectoption.get()

    api_key = '0N5wm6WqCx3i2FHlcaNeO0QfFRheAzFvx5GaHs9K'
    custom_image_url_pattern = r'url": "(.*?)\"'
    custom_title_pattern = r'title": "(.*?)\"'
    custom_explanation_pattern = r'explanation": "(.*?)\"'
    custom_date_pattern = r'date": "(.*?)\"'

    url = f'https://api.nasa.gov/planetary/apod?api_key={api_key}&start_date={start_date}&end_date={formatted_Date}'
    response = requests.get(url)
    if(response.status_code == 200 and data_source == "MySql"):
        api_response = response.json()
        for i in api_response:
            res = str(i)
            result = res.replace("\'","\"")
            image_url = extract_with_pattern(result, custom_image_url_pattern)
            title = extract_with_pattern(result, custom_title_pattern)
            explanation = extract_with_pattern(result, custom_explanation_pattern)
            date = extract_with_pattern(result, custom_date_pattern)
            
            sql = "INSERT INTO project.nasa (image_url, title, explanation, date) VALUES (%s, %s, %s, %s);"
            val = (image_url, title, explanation, date)
            mycursor.execute(sql, val)
            mydb.commit()
            listBox.insert("", "end", values=(date, image_url, title, explanation))

    elif(response.status_code == 200 and data_source == "json"):
        api_response_json = response.json()
        for i in api_response_json:
            res = str(i)
            result = res.replace("\'","\"")
            
            image_url = extract_with_pattern(result, custom_image_url_pattern)
            title = extract_with_pattern(result, custom_title_pattern)
            explanation = extract_with_pattern(result, custom_explanation_pattern)
            date = extract_with_pattern(result, custom_date_pattern)
            
            filename = formatted_Date + '.json'
            with open(filename, 'w') as json_file:
                json.dump(api_response_json, json_file, indent=4)
            listBox.insert("", "end", values=(date, image_url, title, explanation))

l2 = tk.Label(master, text = "Select a date : ")
checkboxLabel = tk.Label(master, text = "Select a Data source : ")
selectoption = tk.StringVar()
check_button1=tk.Radiobutton(master,text="MySQL",variable=selectoption,value="MySql")
check_button2=tk.Radiobutton(master,text="JSON",variable=selectoption,value="json")
click_button=tk.Button(master,text="Retrive Data",command=handler)
e2 = DateEntry(master ,width=30,bg="darkblue",fg="white",year=2022)
 
l2.grid(row = 0, column = 0)
checkboxLabel.grid(row = 1, column = 0)
check_button1.grid(row = 1, column = 1)
check_button2.grid(row = 2, column = 1)
click_button.grid(row = 3, column = 0, pady=0)
e2.grid(row = 0, column = 1)

cols = ('Date', 'Image URL', 'Title', 'Explanation')
listBox = ttk.Treeview(master, columns=cols, show='headings')
for col in cols:
    listBox.heading(col, text=col)    
listBox.grid(row=5, column=0, columnspan=2)

master.mainloop()